In [ ]:
pip install pyspellchecker

     |████████████████████████████████| 2.7MB 17.7MB/s 


In [ ]:
from spellchecker import SpellChecker
import numpy as np
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')
from tensorflow import keras
import random
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

with open('/content/gdrive/My Drive/Datasets/Intent2.json') as file:
    data = json.load(file)

Mounted at /content/gdrive


In [ ]:
data['intents'][0]

{'context': {'clear': False, 'in': '', 'out': 'GreetingUserRequest'},
 'entities': [],
 'entityType': 'NA',
 'extension': {'entities': False, 'function': '', 'responses': []},
 'intent': 'Greeting',
 'responses': ['Hi human, please tell me your GeniSys user',
  'Hello human, please tell me your GeniSys user',
  'Hola human, please tell me your GeniSys user'],
 'text': ['Hi',
  'Hi there',
  'Hola',
  'Hello',
  'Hello there',
  'Hya',
  'Hya there']}

In [ ]:
# Remove Stopwords (Stopwords like 'a', 'the', ... )

StopWordsList = set(stopwords.words('english'))

def remove_stopwords(text):
    return ' '.join([word for word in str(text).split() if word not in StopWordsList])

In [ ]:
# Spleling correction

spell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)

In [ ]:
# Word Tokenizing and define X and Y

words = []      # all of words that are in sentences
all_labels = [] # intents
sentences = []  # input sentences(X)
y = []          # label that match to each sentence

for intent in data['intents']:
    for pattern in intent['text']:
      pattern_stopwords = remove_stopwords(pattern)
      pattern_spelling = remove_stopwords(pattern_stopwords)
      wrds = nltk.word_tokenize(pattern_spelling)
      words.extend(wrds)
      sentences.append(wrds)
      y.append(intent["intent"])
        
    if intent['intent'] not in all_labels:
        all_labels.append(intent['intent'])

In [ ]:
# Stemming

stemmer = LancasterStemmer()

words = [stemmer.stem(w.lower()) for w in words]
words = sorted(list(set(words)))

all_labels = sorted(all_labels)

In [ ]:
# Make bag of words

training = []
output = []

out_empty = [0 for _ in range(len(all_labels))]

for indx, sentence in enumerate(sentences):

    bag = []

    wrds = [stemmer.stem(w.lower()) for w in sentence]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[all_labels.index(y[indx])] = 1

    training.append(bag)
    output.append(output_row)

training = np.array(training)
output = np.array(output)

In [ ]:
# make MLP model

model = keras.models.Sequential()

model.add(keras.layers.Dense(128, activation='relu', input_dim=len(training[0])))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(len(output[0]), activation='softmax'))

model.compile(loss='mse', optimizer='rmsprop', metrics='accuracy')

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 128)               13312     
_________________________________________________________________
dense_7 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_8 (Dense)              (None, 22)                1430      
Total params: 22,998
Trainable params: 22,998
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(training, output, epochs=500, batch_size=16, validation_split=0.2)

Epoch 1/500
8/8 [==============================] - 0s 32ms/step - loss: 4.1425e-04 - accuracy: 0.9912 - val_loss: 2.6455e-09 - val_accuracy: 1.0000
Epoch 2/500
8/8 [==============================] - 0s 8ms/step - loss: 4.1270e-04 - accuracy: 0.9825 - val_loss: 2.6867e-09 - val_accuracy: 1.0000
Epoch 3/500
8/8 [==============================] - 0s 8ms/step - loss: 4.1386e-04 - accuracy: 0.9912 - val_loss: 2.7197e-09 - val_accuracy: 1.0000
Epoch 4/500
8/8 [==============================] - 0s 8ms/step - loss: 4.1178e-04 - accuracy: 0.9825 - val_loss: 2.7349e-09 - val_accuracy: 1.0000
Epoch 5/500
8/8 [==============================] - 0s 8ms/step - loss: 4.1215e-04 - accuracy: 0.9825 - val_loss: 2.7826e-09 - val_accuracy: 1.0000
Epoch 6/500
8/8 [==============================] - 0s 8ms/step - loss: 3.9954e-04 - accuracy: 0.9912 - val_loss: 2.8598e-09 - val_accuracy: 1.0000
Epoch 7/500
8/8 [==============================] - 0s 8ms/step - loss: 4.1786e-04 - accuracy: 0.9825 - val_loss: 2.91

In [ ]:
# Chat with chatbot

# make bag of words for input text
def bag_of_words(s, words):

    bag = [0 for _ in range(len(words))]
    
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return np.array(bag)

# predict intenet and choose response
def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        inp_stopwords = remove_stopwords(inp)
        inp_spelling = remove_stopwords(inp_stopwords)
        results = model.predict(np.array([bag_of_words(inp_spelling, words)]))
        results_index = np.argmax(results)
        tag = all_labels[results_index]

        for tg in data['intents']:
            if tg['intent'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

chat()

Start talking with the bot (type quit to stop)!
You: hello. how are you ?
Hola human, please tell me your GeniSys user
You: what's your name ?
<HUMAN>, what can I do for you?
You: no i want your real name?
GeniSys
You: tell me a joke
David Hasselhoff walks into a bar and says to the barman, 'I want you to call me David Hoff'.  The barman replies 'Sure thing Dave... no hassle'
You: tell me about self-aware..
System says no!
You: can you open the door please?
I’m sorry, I’m afraid I can’t do that!
You: thank you
Happy to help!
You: goodbye
See you later
You: quit
